In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import MLP_generator,test_MLP_generator,createGraphANN
import cPickle

In [2]:
batch_size = 100
seq_len = 16 # same as test set for convenience
learning_rate = 1e-4
epoch = 10

In [3]:
# time dependent list of discrete variables of shape (B,T)
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']
# time independent list of discrete variables of shape of shape (B,)
disc_X = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]

In [5]:
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [6]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = 50

In [7]:
lambdas = np.array([2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) * 5e-4 # regularization coefficient
l1,l2,l3 = 5e-4,5e-4,5e-4

** Build Computational Graph **

In [8]:
inputs,train_op,cost,saver,yhat = createGraphANN(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                        dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

** Training **

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
monitor = 10000

In [11]:
prefix = 'train_final'

In [12]:
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [13]:
for i in range(epoch):
    cost_train = .0
    weight_train = .0
    for j,X_nps in enumerate(MLP_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                              batch_size,seq_len,shuffle=True,downSample=0.5)):
        _,cost_np = sess.run([train_op,cost],dict(zip(inputs,X_nps+[learning_rate])))
        cost_train += cost_np
        weight_train = weight_train + np.sum(X_nps[1])
        
        if j%monitor == 0:       
            print "Train loss:{}".format(np.sqrt(seq_len*batch_size*cost_train/weight_train))
            cost_train = .0
            weight_train = .0

Train loss:1.26114785018
Train loss:0.77401129207
Train loss:0.715394053974
Train loss:0.811131657205
Train loss:0.609291726046
Train loss:0.641365244039
Train loss:0.807697404348
Train loss:0.731763103699
Train loss:0.658220376053
Train loss:0.87544487693


In [14]:
# Save model parameters
saver.save(sess,'ANN_fillin_01')

'ANN_fillin_01'

*** Prediction ***

Load trained model

In [15]:
inputs,train_op,cost,saver,yhat = createGraphANN(None,seq_len,cardinalitys_X,cardinalitys_T,\
                                              dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
saver.restore(sess,'ANN_fillin_01')

INFO:tensorflow:Restoring parameters from ANN_fillin_01


Load test dataset

In [18]:
prefix = 'test_newItem_final'

In [19]:
y_np_newItem = np.loadtxt(prefix+'_Y',dtype=np.int32, delimiter=",") 
Con_np_newItem = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_newItem = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Dis_np_newItem = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

Score test dataset

In [20]:
yhat_newItem, ID_newItem = [],[]
for y,x in test_MLP_generator(y_np_newItem, Con_np_newItem,Dis_np_newItem,X_np_newItem,batch_size):
    ID_newItem.append(np.reshape(y,(-1)))
    yhat_newItem.append(np.reshape(sess.run(yhat,dict(zip(inputs[2:-1],x))),(-1)))
newItem = pd.DataFrame({'id':np.concatenate(ID_newItem),'unit_sales':np.concatenate(yhat_newItem)})    

In [21]:
SI = pd.read_csv('RNN_01.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [22]:
zeroSubmission = pd.read_csv('zeroSubmission_final.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [23]:
pd.concat([SI,newItem,zeroSubmission]).sort_values(['id']).to_csv('submit_final1.csv',index=False)

In [38]:
#submission = pd.read_csv('sample_submission.csv',dtype={'id': 'int32','unit_sales': 'float32'})
#forecastId = SI.id.tolist() + newItem.id.tolist()
#submission[~submission.id.isin(forecastId)].to_csv('zeroSubmission_final.csv',index=False)